In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re

In [2]:
toy_rev = pd.read_csv('Scraped_Car_Review_ferrari.csv',engine='python',index_col=False)
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,on 04/28/17 08:08 AM (PDT),Garrett Stites,2015 Ferrari 458 Italia Convertible Spider 2dr...,The best car around!,This car gets great gas mileage and is the be...,5.00
1,1,on 11/19/11 16:47 PM (PST),debu99,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,keeps on beeing just great,Owning the 612 now over 3 years and using it ...,4.75
2,2,on 06/28/07 22:12 PM (PDT),Arnell Baylet,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,"Incredible Ride, Sticker Shock, Low MPG",Best controllable acceleration ever witnessed...,5.00
3,3,on 05/30/07 10:56 AM (PDT),gregMTU,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,612 Scag The Best!,The engine is strong pulling right up to 7000...,5.00
4,4,on 01/21/07 12:58 PM (PST),Darrel McOnnery,2006 Ferrari 612 Scaglietti Coupe 2dr Coupe (5...,612 Scaglietti! Quietly the Best Ferrari Ever!,"This car, the 612 Scaglietti, is overlooked ...",5.00


In [3]:
import spacy
from tqdm import tqdm
nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)

In [4]:
toy_rev['review']=toy_rev['Review_Title']+toy_rev['Review']

In [5]:
competitors = ['Chevy','chevy','Ford','ford','Nissan','nissan','Honda','honda','Chevrolet','chevrolet','Volkswagen','volkswagen','benz','Benz','Mercedes','mercedes','subaru','Subaru','VW']


In [6]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(toy_rev['review']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(toy_rev['review'][x])) != 0:
        lines = str(toy_rev['review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')       
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])
            
    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
toy_rev['compound_nouns'] = comp_terms
toy_rev['aspect_keywords'] = aspect_terms
toy_rev.head()

100%|██████████| 260/260 [00:04<00:00, 63.31it/s]


,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,review,compound_nouns,aspect_keywords
0,0,on 04/28/17 08:08 AM (PDT),Garrett Stites,2015 Ferrari 458 Italia Convertible Spider 2dr...,The best car around!,This car gets great gas mileage and is the be...,5.00,The best car around! This car gets great gas m...,"[(gas mileage, mileage)]","[best car, great gas mileage]"
1,1,on 11/19/11 16:47 PM (PST),debu99,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,keeps on beeing just great,Owning the 612 now over 3 years and using it ...,4.75,keeps on beeing just great Owning the 612 now ...,"[(evey day, day), (day traffic, traffic), (pai...","[ever owned, best car, only problem]"
2,2,on 06/28/07 22:12 PM (PDT),Arnell Baylet,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,"Incredible Ride, Sticker Shock, Low MPG",Best controllable acceleration ever witnessed...,5.00,"Incredible Ride, Sticker Shock, Low MPG Best c...","[(Seat comfort, comfort), (Ferrari lineup, lin...","[ever witnessed, owned prior, Incredible Ride,..."
3,3,on 05/30/07 10:56 AM (PDT),gregMTU,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,612 Scag The Best!,The engine is strong pulling right up to 7000...,5.00,612 Scag The Best! The engine is strong pullin...,[],[enjoyable car]
4,4,on 01/21/07 12:58 PM (PST),Darrel McOnnery,2006 Ferrari 612 Scaglietti Coupe 2dr Coupe (5...,612 Scaglietti! Quietly the Best Ferrari Ever!,"This car, the 612 Scaglietti, is overlooked ...",5.00,612 Scaglietti! Quietly the Best Ferrari Ever!...,[],"[however performs, ever owned]"


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [8]:
import operator
sentiment = []
for i in range(len(toy_rev)):
    score_dict={'pos':0,'neg':0,'neu':0}
    if len(toy_rev['aspect_keywords'][i])!=0: 
        for aspects in toy_rev['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
        #score_dict['neu'] += sent['neu']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
toy_rev['sentiment'] = sentiment
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,review,compound_nouns,aspect_keywords,sentiment
0,0,on 04/28/17 08:08 AM (PDT),Garrett Stites,2015 Ferrari 458 Italia Convertible Spider 2dr...,The best car around!,This car gets great gas mileage and is the be...,5.00,The best car around! This car gets great gas m...,"[(gas mileage, mileage)]","[best car, great gas mileage]",pos
1,1,on 11/19/11 16:47 PM (PST),debu99,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,keeps on beeing just great,Owning the 612 now over 3 years and using it ...,4.75,keeps on beeing just great Owning the 612 now ...,"[(evey day, day), (day traffic, traffic), (pai...","[ever owned, best car, only problem]",pos
2,2,on 06/28/07 22:12 PM (PDT),Arnell Baylet,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,"Incredible Ride, Sticker Shock, Low MPG",Best controllable acceleration ever witnessed...,5.00,"Incredible Ride, Sticker Shock, Low MPG Best c...","[(Seat comfort, comfort), (Ferrari lineup, lin...","[ever witnessed, owned prior, Incredible Ride,...",pos
3,3,on 05/30/07 10:56 AM (PDT),gregMTU,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,612 Scag The Best!,The engine is strong pulling right up to 7000...,5.00,612 Scag The Best! The engine is strong pullin...,[],[enjoyable car],pos
4,4,on 01/21/07 12:58 PM (PST),Darrel McOnnery,2006 Ferrari 612 Scaglietti Coupe 2dr Coupe (5...,612 Scaglietti! Quietly the Best Ferrari Ever!,"This car, the 612 Scaglietti, is overlooked ...",5.00,612 Scaglietti! Quietly the Best Ferrari Ever!...,[],"[however performs, ever owned]",pos


In [9]:
int_sent = []
for sent in toy_rev['sentiment']:
    if sent is 'NaN':
        int_sent.append('NaN')
    elif sent is 'pos':
        int_sent.append('1')
    else:
        int_sent.append('0')
toy_rev['int_sent'] = int_sent
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,review,compound_nouns,aspect_keywords,sentiment,int_sent
0,0,on 04/28/17 08:08 AM (PDT),Garrett Stites,2015 Ferrari 458 Italia Convertible Spider 2dr...,The best car around!,This car gets great gas mileage and is the be...,5.00,The best car around! This car gets great gas m...,"[(gas mileage, mileage)]","[best car, great gas mileage]",pos,1
1,1,on 11/19/11 16:47 PM (PST),debu99,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,keeps on beeing just great,Owning the 612 now over 3 years and using it ...,4.75,keeps on beeing just great Owning the 612 now ...,"[(evey day, day), (day traffic, traffic), (pai...","[ever owned, best car, only problem]",pos,1
2,2,on 06/28/07 22:12 PM (PDT),Arnell Baylet,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,"Incredible Ride, Sticker Shock, Low MPG",Best controllable acceleration ever witnessed...,5.00,"Incredible Ride, Sticker Shock, Low MPG Best c...","[(Seat comfort, comfort), (Ferrari lineup, lin...","[ever witnessed, owned prior, Incredible Ride,...",pos,1
3,3,on 05/30/07 10:56 AM (PDT),gregMTU,2006 Ferrari 612 Scaglietti Coupe F1 2dr Coupe...,612 Scag The Best!,The engine is strong pulling right up to 7000...,5.00,612 Scag The Best! The engine is strong pullin...,[],[enjoyable car],pos,1
4,4,on 01/21/07 12:58 PM (PST),Darrel McOnnery,2006 Ferrari 612 Scaglietti Coupe 2dr Coupe (5...,612 Scaglietti! Quietly the Best Ferrari Ever!,"This car, the 612 Scaglietti, is overlooked ...",5.00,612 Scaglietti! Quietly the Best Ferrari Ever!...,[],"[however performs, ever owned]",pos,1


In [ ]:
for i,j,z in zip(toy_rev['review'],toy_rev['sentiment'],toy_rev['aspect_keywords']) :
    print("Review:" + " "+ str(i) + "Sentiment:" + " " + str(j) +"\n "+"Aspect words:" + str(z))

In [ ]:
toy_rev.to_csv('sent.csv', index=False)

In [ ]:
import math
pos = []
for i in range(len(toy_rev)):
    if not math.isnan(toy_rev['Rating'][i]):
        if int(toy_rev['Rating'][i])>3:
            pos.append('1')
        else:
            pos.append('0')
    else:
        pos.append('0')
toy_rev['Positive Review'] = pos
toy_rev.head()

In [ ]:
d = {'sent':toy_rev['Positive Review'],'sent_pred':toy_rev['int_sent']}
metric_df = pd.DataFrame(data=d)
metric_df.head()

In [ ]:
len(metric_df.sent)

In [ ]:
metric_df = metric_df[metric_df.sent_pred != 'NaN']
len(metric_df.sent)

In [ ]:
from sklearn.metrics import accuracy_score,auc,f1_score,recall_score,precision_score
print('accuracy')
print(accuracy_score(metric_df.sent, metric_df.sent_pred))
print('f1 score')
print(f1_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('recall')
print(recall_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('precision')
print(precision_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))